# Integrating snATAC-seq ortholog peak matrix
## human, macaque, and mouse caudate/caudoputamen

Notes on ortholog peaks notes: 
- reproduced peaks called in human, macaque, and mouse independently
- peaks pairwise mapped to orthologs via HALPER
    - keep only peaks mappable to all species
- ortholog peak set is the union of mappable orthologs from all species
    - not all ortholog peaks were reproducible peaks mapped in every dataset
    - unioning done in hg38 coordinates
    - this new peak set mapped back to mouse and macaque

- count matrix comes from union of mappable orthologs detected in any data set
    - the peaks in human, mouse, macaque used to get feature matrix in each species
    - feature matrix linked by peak name from hg38 coordinates
    - implies mappable ortholog changing chromatin accessibility between species

Notes on integration:
- perform integration with about same number of cells in each group
    - here selected representative human and macaque subject
    - also using a guide tree in iterative merging
- predefine the features during anchor finding step

In [ ]:
PROJDIR=file.path('../../../data/raw_data/cross_species_peak_orthologs')

#######################################
### set up libraries and functions ####
ss <- function(x, pattern, slot = 1, ...) { 
  sapply(strsplit(x = x, split = pattern, ...), '[', slot) }
options(stringsAsFactors = F, repr.plot.width=14, repr.plot.height=6.5)
suppressMessages(library(Signac)); suppressMessages(library(Seurat))
suppressMessages(library(GenomicRanges))

source('../hal_scripts/narrowPeakFunctions.R')
parallel::detectCores()

In [ ]:
# # set up future for parallelization
library(future)
library(future.apply)
plan("sequential")
options(future.globals.maxSize = 180 * 1024^3)

# 1) visualize unintegrated species clusters

In [ ]:
## load the ortholog peak seurat object
saveRDS_fn = file.path(PROJDIR, 'rdas', 'multispeciesMergedSeurat.rds')
obj_seurat = readRDS(file = saveRDS_fn)

##  TF-IDF, SVD, and UMAP already performed on 'peaks' matrix
obj_seurat

In [ ]:
### subset to just humans ###
cells = WhichCells(obj_seurat, expression = Sample %in% c("14_1018.CAUD", 'CAUD_WS1H_STA682A131'))
obj_seurat = subset(obj_seurat, cells = cells)

## show cells per Species
table(obj_seurat@meta.data$Sample)

## show cell clusters per sample
table(obj_seurat@meta.data$Clusters2, obj_seurat@meta.data$Sample)

In [ ]:
keepRegion = rownames(obj_seurat)[which(rowSums(assays(obj_seurat)$peaks) > 0)]

In [ ]:
length(keepRegion); head(keepRegion)

In [ ]:
## recompute TFIDF, SVD, and UMAP
obj_seurat <- RunTFIDF(obj_seurat, verbose = FALSE)
obj_seurat <- RunSVD(obj_seurat, verbose = FALSE)
obj_seurat <- RunUMAP(obj_seurat, reduction = 'lsi', dims = 2:30, verbose = FALSE)
gc()

In [ ]:
p_unintegrated_species = 
    DimPlot(object = obj_seurat, label = FALSE, group.by = 'Sample', cols = 'Dark2') + 
    ggplot2::ggtitle("Unintegrated")

p_unintegrated_clusters2 = 
    DimPlot(object = obj_seurat, label = TRUE, group.by = 'Clusters2', cols = 'Paired') + 
    ggplot2::ggtitle("Unintegrated")

p_unintegrated_species + p_unintegrated_clusters2

# 2) integration with Seurat Intergration

In [ ]:
## split seurat object up by species
obj_seurat.list = SplitObject(obj_seurat, split.by = 'Sample')
print(names(obj_seurat.list))
obj_seurat.list = lapply(obj_seurat.list, function(x){
  x <- RunTFIDF(x, verbose = FALSE)
#   x <- FindTopFeatures(x, min.cutoff = 'q5')
  x <- RunSVD(x, verbose = FALSE)
})

# features <- SelectIntegrationFeatures(object.list = obj_seurat.list, nfeatures = 10000)
features <- rownames(obj_seurat)

In [ ]:
# using 14_1018.CAUD as reference
ref = which(names(obj_seurat.list) =="14_1018.CAUD") 

# find integration anchors between species, using all features
anchors <- FindIntegrationAnchors(
        object.list = obj_seurat.list, reduction = 'cca', anchor.features = features,
        reference = c(ref), k.filter = NA, assay = rep('peaks', length(obj_seurat.list)))
gc()

In [ ]:
# integrate data and create a new merged object
integrated <- IntegrateData(anchors, dims = 2:30, preserve.order = TRUE)

# we now have a "corrected" TF-IDF matrix, and can run LSI again on this corrected matrix
integrated <- RunSVD(integrated, n = 30, reduction.name = 'integratedLSI', verbose = FALSE)
integrated <- RunUMAP(integrated, dims = 2:30, reduction = 'integratedLSI', verbose = FALSE)

# 3) compare integrated snATAC-seq cell types

In [ ]:
# plot embeddings
p_seuratIntegration_species = 
    DimPlot(object = integrated, label = FALSE, group.by = 'Sample', cols = 'Dark2') +
    ggplot2::ggtitle('Seurat CCA Integration')

p_seuratIntegration_clusters2 = 
    DimPlot(object = integrated, label = TRUE, group.by = 'Clusters2', cols = 'Paired') +
    ggplot2::ggtitle('Seurat CCA Integration')

p_seuratIntegration_species + p_seuratIntegration_clusters2

In [ ]:
DimPlot(object = integrated, label = TRUE, group.by = 'Clusters2', cols = 'Paired',split.by = 'Sample') +
    ggplot2::ggtitle('Seurat CCA Integration')

In [ ]:
rm(obj_seurat.list, obj_seurat, anchors); gc()

In [ ]:
integrated
object.size(integrated) / 1024^3

In [ ]:
## save the seurat object
integratedRDS_fn = file.path(PROJDIR,'rdas','mergedMultiSpeciesSeuratCCAsubsetHumMac.rds')
saveRDS(integrated, file = integratedRDS_fn)